In [1]:
!pip install xlsxwriter

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install html5lib

Defaulting to user installation because normal site-packages is not writeable


In [5]:
import pandas as pd
import numpy as np
import os
from io import BytesIO
import warnings
warnings.filterwarnings("ignore", "This pattern is interpreted as a regular expression")

# ================================
# CONFIGURATION
# ================================
SIGNED_HTML_PATH = r'C:/Users/Testing Rename nalng/Documents/Sports Interactive/Football Manager 2024/Frosinone Signings Available S1 Winter.html' #Transfers
LOANS_HTML_PATH = r'C:/Users/Testing Rename nalng/Documents/Sports Interactive/Football Manager 2024/Frosinone Loans Available S1 Winter.html' #Loans
UNIVERSAL_HTML_PATH = r'C:/Users/Testing Rename nalng/Documents/Sports Interactive/Football Manager 2024/Frosinone Universe S1 Winter.html'
OUTPUT_PATH = r'C:/Users/Testing Rename nalng/Desktop/Frosinone_Season_1_Winter.xlsx'


TEXT_COLUMNS = [
    'UID', 'Name', 'Rec', 'EU National', 'Position', 'Pros',
    'Preferred Foot', 'Inf', 'Transfer Value', 'Nat', 'Division', 'Club', 'Personality'
]

PERCENTAGE_COLUMNS = ['Sv %', 'OP-Cr %', 'Hdr %', 'Conv %', 'Pas %', 'Cr C/A', 'Tck R', 'Pens Saved Ratio', 'Pen/R', 'Shot %']

# ================================
# LEAGUE POWER RATINGS (0 - 100)
# ================================
LEAGUE_POWER = {
    'Premier League': 92.6, #England
    'LALIGA EA SPORTS': 87, #Spain
    'Serie A TIM': 87, #Italy
    'Bundesliga': 86.3, #Germany
    'Ligue 1 Uber Eats': 85.5, #France
    'Sky Bet Championship': 80.9, #England 2nd Division
    'Jupiler Pro League': 80.5, #Belgium
    'Liga Portugal Betclic': 79.8, #Portugal
    'Brasileirão Assaí Série A' : 79.4, #Brazil
    'BrasileirÃ£o AssaÃ­ SÃ©rie A': 79.4, #Brazil
    'Eredivisie': 78.8, #Netherlands
    'Liga Profesional de Fútbol': 78.6, #Argentina
    'Liga Profesional de FÃºtbol': 78.6, #Argentina
    'Major League Soccer': 78.5, #USA
    'Liga MX': 78.5, #Mexico
    'Meiji Yasuda J1 League': 77.9,  #Japan
    'SuperSport Hrvatska nogometna liga': 77.8, #Croatia
    'PKO Ekstraklasa': 77.6,  #Poland
    '3F Superliga': 77.6, #Denmark
    'Serie BKT': 76.4,  #Italy 2nd Division
    'Allsvenskan': 76.3, #Sweden
    'Spor Toto Süper Lig': 76.2, #Turkey
    'LALIGA HYPERMOTION': 76.2,  #Spain 2nd Division
    '2. Bundesliga': 76.2, #German 2nd Division
    'RPL': 76.1, #Russia
    'Admiral Bundesliga': 76.1, #Austria
    'Raiffeisen Super League': 76.1,
    'Eliteserien': 75.9, #Norway
    'Fortuna liga': 75.5, #Czech
    'Roshn Saudi League': 75.1, #Saudi
    
    'Others': 50  # Default value for unlisted leagues
}

# ================================
# LOAD HTML FILES
# ================================
if not os.path.exists(SIGNED_HTML_PATH):
    raise FileNotFoundError(f"Signed players HTML not found: {SIGNED_HTML_PATH}")

if not os.path.exists(UNIVERSAL_HTML_PATH):
    raise FileNotFoundError(f"Universal players HTML not found: {UNIVERSAL_HTML_PATH}")

if not os.path.exists(LOANS_HTML_PATH):
    raise FileNotFoundError(f"Loan players HTML not found: {LOANS_HTML_PATH}")

with open(SIGNED_HTML_PATH, 'r', encoding='utf-8') as file:
    signed_tables = pd.read_html(file)
df_signed = signed_tables[0].copy()

with open(UNIVERSAL_HTML_PATH, 'r', encoding='utf-8') as file:
    universal_tables = pd.read_html(file)
df_universal = universal_tables[0].copy()

with open(LOANS_HTML_PATH, 'r', encoding='utf-8') as file:
    loans_tables = pd.read_html(file)
df_loans = loans_tables[0].copy()


In [6]:

# ================================
# UID‑BASED MERGE AND LABEL  ✅ Fixed version
# ================================

def normalize_uid(df):
    df = df.copy()
    # Convert to float → Int64 (nullable integer) → string
    df['UID'] = pd.to_numeric(df['UID'], errors='coerce')  # force numeric, NaNs stay if bad
    df['UID'] = df['UID'].astype('Int64')                  # keep NaNs clean
    df['UID'] = df['UID'].astype(str).str.strip()          # final string
    return df

# 1️⃣ Normalize UIDs
df_signed    = normalize_uid(df_signed)
df_universal = normalize_uid(df_universal)
df_loans = normalize_uid(df_loans)

# 2️⃣ Add the signability flags
df_signed['Signability']    = 'Available for Transfer'
df_universal['Signability'] = 'Not Transferrable'
df_loans['Signability'] = 'Available on Loan'


# 3️⃣ Concatenate signables first, drop duplicate UIDs

df = (
    pd.concat([df_signed, df_loans, df_universal], ignore_index=True)
      .drop_duplicates(subset='UID', keep='first')
      .reset_index(drop=True)
)

# ================================
# CLEANING & CONVERSION
# ================================
def clean_and_convert_data(df):
    def convert_percentage_to_float(df, column):
        if column in df.columns:
            df[column] = (df[column].astype(str)
                          .str.replace('%', '')
                          .replace('-', np.nan)
                          .astype(float))
        return df

    # Convert minutes
    if 'Mins' in df.columns:
        df['Mins'] = pd.to_numeric(df['Mins'], errors='coerce')
        df = df[df['Mins'] >= 900].copy()

    # Convert percentage stats
    for col in PERCENTAGE_COLUMNS:
        df = convert_percentage_to_float(df, col)

    # Handle 'Dist/90' - e.g., "7.3mi"
    if 'Dist/90' in df.columns:
        df['Dist/90'] = (
            df['Dist/90']
            .astype(str)
            .str.extract(r'([\d.]+)')[0]
        )

        # Drop only if the string was completely missing
        df['Dist/90'] = pd.to_numeric(df['Dist/90'], errors='coerce')

    print("Dist/90 (after cleaning):")
    print(df['Dist/90'].describe())
    print(df['Dist/90'].dropna().head(10))






    

    # Convert all other numerical columns (except text & signability)
    for col in df.columns:
        if col not in TEXT_COLUMNS + ['Signability']:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # Handle 'Dist/90' - e.g., "7.3mi"
    if 'Dist/90' in df.columns:
        df['Dist/90'] = (
            df['Dist/90']
            .astype(str)
            .str.extract(r'([\d.]+)')[0]
        )

        # Drop only if the string was completely missing
        df['Dist/90'] = pd.to_numeric(df['Dist/90'], errors='coerce')

    print("Dist/90 (after cleaning):")
    print(df['Dist/90'].describe())
    print(df['Dist/90'].dropna().head(10))


    # Add league strength multiplier
    def get_league_multiplier(division):
        return LEAGUE_POWER.get(division, LEAGUE_POWER['Others']) / 100.0

    df['League Multiplier'] = df['Division'].apply(get_league_multiplier)
    return df

# ✅ Apply cleaning
df = clean_and_convert_data(df)

# ================================
# CONVERT RAW STATS TO PER 90
# ================================
RAW_STATS_TO_PER90 = {
    "Yel": "Yellow/90",
    "Red": "Red/90",
    "Fls": "FoulsMade/90",
    "FA": "FoulsAgainst/90",
    "Off": "Offsides/90",
    "Gl Mst": "Gl Mst/90",
    "Goals Outside Box": "Goals Outside Box/90",
    "FK Shots": "FKShots/90"
}

for raw_stat, per90_stat in RAW_STATS_TO_PER90.items():
    if raw_stat in df.columns:
        df[raw_stat] = pd.to_numeric(df[raw_stat], errors='coerce')
        df['Mins'] = pd.to_numeric(df['Mins'], errors='coerce')
        df[per90_stat] = (df[raw_stat] / (df['Mins'] / 90)).fillna(0)
    else:
        print(f"Warning: Raw stat '{raw_stat}' not found in DataFrame")

print(df['Dist/90'].dropna().unique()[:50])
print(df['Dist/90'].dtype)


Dist/90 (after cleaning):
count    14758.000000
mean         4.645209
std          3.310233
min          0.000000
25%          0.200000
50%          6.700000
75%          7.500000
max          8.800000
Name: Dist/90, dtype: float64
0    6.8
1    8.0
2    7.5
3    7.8
4    7.9
5    3.6
6    7.1
7    7.7
8    8.2
9    0.0
Name: Dist/90, dtype: float64
Dist/90 (after cleaning):
count    14758.000000
mean         4.645209
std          3.310233
min          0.000000
25%          0.200000
50%          6.700000
75%          7.500000
max          8.800000
Name: Dist/90, dtype: float64
0    6.8
1    8.0
2    7.5
3    7.8
4    7.9
5    3.6
6    7.1
7    7.7
8    8.2
9    0.0
Name: Dist/90, dtype: float64
[6.8 8.  7.5 7.8 7.9 3.6 7.1 7.7 8.2 0.  7.3 3.8 3.  7.  6.9 1.4 7.2 0.4
 7.4 8.1 8.5 3.9 0.9 0.8 3.3 6.4 3.2 0.1 8.6 7.6 1.  8.3 4.  3.7 6.7 2.8
 5.7 0.5 1.8 1.1 2.6 0.6 2.9 3.4 1.7 1.2 3.5 1.6 8.4 5.3]
float64


In [10]:
# Show overall info
print("==== DataFrame Info ====")
print(df.info())

print("\n==== Column Summary ====")
for col in df.columns:
    print(f"\n📌 Column: {col}")
    print(f"Type: {df[col].dtype}")
    print(f"Missing: {df[col].isna().sum()} / {len(df)}")
    
    if df[col].dtype == 'object':
        unique_vals = df[col].dropna().unique()
        print(f"Unique Values: {len(unique_vals)}")
        print("Sample:", unique_vals[:10])
    
    elif pd.api.types.is_numeric_dtype(df[col]):
        print(df[col].describe())

    else:
        print("Sample:", df[col].dropna().head(5).tolist())

==== DataFrame Info ====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18625 entries, 0 to 18624
Data columns (total 88 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   EU National           18625 non-null  object 
 1   Name                  18625 non-null  object 
 2   UID                   18625 non-null  object 
 3   Mins                  18625 non-null  int64  
 4   Position              18625 non-null  object 
 5   Shot %                16941 non-null  float64
 6   Personality           867 non-null    object 
 7   Age                   18625 non-null  int64  
 8   Preferred Foot        18625 non-null  object 
 9   Rec                   18625 non-null  object 
 10  Club                  18625 non-null  object 
 11  Transfer Value        18625 non-null  object 
 12  Sv %                  1667 non-null   float64
 13  OP-Cr %               13122 non-null  float64
 14  Off                   18625 non-null  int64  

In [7]:
if df['Dist/90'].isna().any():
    print("⚠️ Warning: Missing or zero values found in Dist/90")

In [8]:
# ================================
# DATA PREPROCESSING
# ================================
if 'Mins' in df.columns:
    df['Mins'] = pd.to_numeric(df['Mins'], errors='coerce')
    df = df[df['Mins'] >= 900].copy()

# Create composite metrics before scaling
df['Intensity'] = df["Sprints/90"] / df["Dist/90"].replace(0, 1)  # Avoid division by zero
df['NetPoss'] = df["Poss Won/90"] - df["Poss Lost/90"]

# Define numeric columns to scale (excluding text columns and Signability)
TEXT_COLUMNS = ['EU National', 'Name', 'UID', 'Position', 'Personality', 'Preferred Foot', 
                'Rec', 'Club', 'Transfer Value', 'Division', 'Nat', 'Inf', 'Signability']
                
numeric_columns = [col for col in df.columns 
                   if col not in TEXT_COLUMNS 
                   and pd.api.types.is_numeric_dtype(df[col])]

# Scale numeric columns (0-1)
for col in numeric_columns:
    if col == 'Age':
        continue  # Skip scaling for Age column
    if df[col].nunique() > 1:  # Only scale if there's variation
        min_val = df[col].min()
        max_val = df[col].max()
        df[col] = (df[col] - min_val) / (max_val - min_val)
    else:
        df[col] = 0.5  # Default value for constant columns

# Add league multiplier (no scaling needed)
def get_league_multiplier(division):
    return LEAGUE_POWER.get(division, LEAGUE_POWER['Others']) / 100.0

df['League Multiplier'] = df['Division'].apply(get_league_multiplier)

# ================================
# UPDATED ARCHETYPE FORMULAS (SCALED)
# ================================
ARCHETYPE_FORMULAS = {
    "Sweeper Keeper": {
        "filter": df['Position'].str.contains("GK", case=False, na=False),
        "formula": lambda d: (
            0.80 * d["xGP/90"] * d["Sv %"] +
            0.05 * d["Pas %"] +
            0.05 * ((d["K Tck/90"] + d["Tck/90"]) * d["Tck R"])/2 +  # Average of 2 metrics
            0.05 * (1 - d["Gl Mst/90"]) +  # Inverted scaling
            0.05 * d["Pens Saved Ratio"]
        ),
        "label": "SK Rating"
    },
    
    "Central Defender": {
        "filter": df['Position'].str.contains(r"^D\s*\(([RLC]*C[RLC]*)\)", regex=True, na=False),
        "formula": lambda d: (
            0.80 * (d["Clr/90"] + d["Int/90"] + d["Blk/90"] + 
                    d["Shts Blckd/90"] + d["Hdrs W/90"] + 
                    ((d["K Tck/90"] + d["Tck/90"]) * d["Tck R"]))/7 +  # Average of 7 metrics
            0.05 * d["Pas %"] +
            0.05 * (1 - d["Gl Mst/90"]) +  # Inverted scaling
            0.05 * (d["Drb/90"] + d["Pr passes/90"])/2 +  # Average of 2 metrics
            0.05 * (d["NetPoss"] + 1)/2  # Scaled to 0-1 range
        ),
        "label": "CD Rating"
    },
    
    "Fullback": {
        "filter": df['Position'].str.contains(r"^(D|WB)\s*\((R|L|RL|RLC)\)", regex=True, na=False),
        "formula": lambda d: (
            0.80 * d["xA/90"] +
            0.05 * (d["Clr/90"] + d["Int/90"] + d["Blk/90"] + 
                    d["Shts Blckd/90"] + d["Hdrs W/90"] + 
                    ((d["K Tck/90"] + d["Tck/90"]) * d["Tck R"]))/7 +  # Average of 7 metrics
            0.05 * d["OP-Crs C/90"] * d["OP-Cr %"] +
            0.05 * (d["Drb/90"] + d["Pr passes/90"])/2 +  # Average of 2 metrics
            0.05 * d["Intensity"]
        ),
        "label": "FB Rating"
    },
    
    "Defensive Midfielder": {
        "filter": df['Position'].str.contains(r"^DM$", regex=True, na=False),
        "formula": lambda d: (
            0.80 * (d["Clr/90"] + d["Int/90"] + d["Blk/90"] + 
                    d["Shts Blckd/90"] + d["Hdrs W/90"] + 
                    ((d["K Tck/90"] + d["Tck/90"]) * d["Tck R"]))/7 +  # Average of 7 metrics
            0.10 * d["Pr passes/90"] * d["Pas %"] +
            0.05 * (1 - d["FoulsMade/90"]) +  # Inverted scaling
            0.05 * (d["NetPoss"] + 1)/2  # Scaled to 0-1 range
        ),
        "label": "DM Rating"
    },
    
    "Winger": {
        "filter": df['Position'].str.contains(r"^AM\s*\((L|R|RL|RLC)\)", regex=True, na=False),
        "formula": lambda d: (
            0.80 * d["xA/90"] +
            0.05 * (d["Drb/90"] + d["Pr passes/90"])/2 +  # Average of 2 metrics
            0.05 * d["Intensity"] +
            0.05 * d["NP-xG/90"] * d["Conv %"] * d["Shot/90"] * d["Shot %"] * d["xG/shot"] +
            0.05 * (d["NetPoss"] + 1)/2  # Scaled to 0-1 range
        ),
        "label": "W Rating"
    },
    
    "Attacking Midfielder": {
        "filter": df['Position'].str.contains(r"^AM\s*\((C|RLC)\)", regex=True, na=False),
        "formula": lambda d: (
            0.80 * d["NP-xG/90"] * d["Conv %"] * d["Shot/90"] * d["Shot %"] * d["xG/shot"] + 
            0.05 * (d["Drb/90"] + d["Pr passes/90"])/2 +  # Average of 2 metrics
            0.05 * d["Intensity"] +
            0.05 * d["xA/90"] +
            0.05 * (d["NetPoss"] + 1)/2  # Scaled to 0-1 range
        ),
        "label": "AM Rating"
    },
    
    "Striker": {
        "filter": df['Position'].str.contains(r"ST", regex=True, na=False),
        "formula": lambda d: (
            0.80 * d["NP-xG/90"] * d["Conv %"] * d["Shot/90"] * d["Shot %"] * d["xG/shot"] + 
            0.10 * d["Gls/90"] +
            0.05 * d["xA/90"] +
            0.05 * (d["NetPoss"] + 1)/2
        ),
        "label": "ST Rating"
    }
}



# ================================
# OUTPUT TO EXCEL WITH SUMMARY SHEET
# ================================
output_excel = BytesIO()

with pd.ExcelWriter(output_excel, engine='xlsxwriter', engine_kwargs={'options': {'strings_to_urls': False}}) as writer:
    workbook = writer.book
    unicode_format = workbook.add_format({'font_name': 'Arial Unicode MS', 'valign': 'vcenter'})
    
    # First create the summary sheet at the beginning
    worksheet = workbook.add_worksheet('Player Archetype Summary')
    writer.sheets['Player Archetype Summary'] = worksheet
    
    # Dictionary to store archetype data for summary
    summary_data = []
    
    # Process all archetypes
    for role, config in ARCHETYPE_FORMULAS.items():
        role_df = df[config["filter"]].copy()
        if role_df.empty:
            continue

        # Calculate ratings
        role_df[config["label"]] = config["formula"](role_df)
        adjusted_label = f"Adjusted {config['label']}"
        role_df[adjusted_label] = role_df[config["label"]] * role_df['League Multiplier']
        
        # Calculate percentile for each player within this archetype
        role_df['Percentile'] = role_df[adjusted_label].rank(pct=True)
        
        # Store data for summary
        top_players = role_df[role_df['Percentile'] > 0.95][['UID', 'Name', 'Percentile']].copy()
        top_players['Archetype'] = role
        summary_data.append(top_players)
        
        # Write archetype sheet
        role_df['Ranking'] = role_df[adjusted_label].rank(method='min', ascending=False)
        
        output_cols = [
            'UID', 'Name', 'Age', 'Personality', 'Signability', 'EU National', 'Position', 'Preferred Foot',
            'Transfer Value', 'Nat', 'Division', 'Club',
            config["label"], adjusted_label, 'Percentile', 'Ranking', 'League Multiplier'
        ]
        
        result_df = role_df[output_cols].copy().sort_values(by=adjusted_label, ascending=False)
        result_df.to_excel(writer, sheet_name=role, index=False)
        
        # Format archetype sheet
        archetype_sheet = writer.sheets[role]
        for i, col in enumerate(output_cols):
            archetype_sheet.set_column(i, i, 20 if col != 'Name' else 30, unicode_format)
        
        # Format percentile as percentage
        percent_format = workbook.add_format({'num_format': '0.0%'})
        percentile_col_idx = output_cols.index('Percentile')
        archetype_sheet.set_column(percentile_col_idx, percentile_col_idx, 12, percent_format)
    
    # ================================
    # BUILD SUMMARY SHEET DATA
    # ================================
    if summary_data:
        all_archetypes = pd.concat(summary_data).sort_values(['UID', 'Percentile'], ascending=[True, False])
        
        # Group and format archetypes
        def format_archetypes(group):
            return ", ".join(f"{row['Archetype']} ({row['Percentile']:.1%})" for _, row in group.iterrows())
        
        player_summary = (
            all_archetypes.groupby(['UID', 'Name'])
            .apply(format_archetypes)
            .reset_index(name='Top Archetypes (>95%)')
        )
        
        # Add additional info
        player_summary = player_summary.merge(
            df[['UID', 'Position', 'Age', 'Nat', 'Club', 'Division', 'Personality', 'Signability', 'Transfer Value']],
            on='UID', how='left'
        )
        
        # Final columns and sorting
        player_summary = player_summary[[
            'UID', 'Name', 'Position', 'Club', 'Division', 
            'Signability', 'Transfer Value', 'Age', 'Nat', 'Personality', 'Top Archetypes (>95%)'
        ]]
        player_summary['Archetype Count'] = player_summary['Top Archetypes (>95%)'].str.count(',') + 1
        player_summary = player_summary.sort_values(['Archetype Count', 'Name'], ascending=[False, True])
        
        # Write to summary sheet
        player_summary.to_excel(writer, sheet_name='Player Archetype Summary', index=False)
        
        # Format summary sheet
        summary_sheet = writer.sheets['Player Archetype Summary']
        summary_sheet.set_column('A:A', 15)  # UID
        summary_sheet.set_column('B:B', 30)  # Name
        summary_sheet.set_column('C:C', 15)  # Position
        summary_sheet.set_column('D:D', 25)  # Club
        summary_sheet.set_column('E:E', 25)  # Division
        summary_sheet.set_column('F:F', 15)  # Signability
        summary_sheet.set_column('G:G', 15)  # Transfer Value
        summary_sheet.set_column('H:H', 60)  # Age
        summary_sheet.set_column('I:I', 60)  # Nationality
        summary_sheet.set_column('J:J', 60)  # Personality
        summary_sheet.set_column('K:K', 60)  # Top Archetypes

# ================================
# SAVE FILE #JUST A COMMENT
# ================================
try:
    with open(OUTPUT_PATH, 'wb') as output_file:
        output_file.write(output_excel.getbuffer())
    print(f"\n✅ Success! Output file saved:\n{OUTPUT_PATH}")
    print(f"Includes {len(ARCHETYPE_FORMULAS)} archetype sheets + summary sheet")
except Exception as e:
    print(f"\n❌ Error saving file: {e}")
    print("Please check directory permissions or disk space.")


C:\Users\Testing Rename nalng\AppData\Local\Temp\ipykernel_9964\779018130.py:201: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(format_archetypes)



✅ Success! Output file saved:
C:/Users/Testing Rename nalng/Desktop/Frosinone_Season_1_Winter.xlsx
Includes 7 archetype sheets + summary sheet
